In [2]:
import pandas as pd

In [4]:
players = pd.read_csv("../staging/transformed/players.csv")
print(players.head())

         Player        Country   Age ResidencyFormer                 Team  \
0        Pantsu         Sweden   NaN             NaN                  NaN   
1  Paolocannone          Italy  29.0          Europe                QLASH   
2        Paopao          China   NaN             NaN                  NaN   
3          PAPA  United States  20.0             NaN  Winthrop University   
4        Papace       Portugal   NaN          Europe                  NaN   

       Residency      Role                     FavChamps RoleLast  IsRetired  \
0         Europe       Mid  Twisted Fate,Kassadin,Veigar      Mid          1   
1           EMEA  Streamer              Blitzcrank,Janna  Support          0   
2          China   Support                           NaN  Support          0   
3  North America       Mid         Irelia,Akali,Kassadin      Mid          0   
4           EMEA       Bot                           NaN      Bot          0   

   ToWildrift                                AllName  
0